In [3]:
import os
from langchain_together import Together
llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0,
    max_tokens=256,
    top_k=1,
    together_api_key=os.getenv("TOGETHER_API_KEY")
)

In [4]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/structured-chat-agent")

In [5]:
prompt[0].prompt.template = prompt[0].prompt.template[0:394] + "```\n\n\nFor example, if you want to use a tool to get an image's caption, your $JSON_BLOB might look like this:\n\n```\n{{\n    'action': 'image_captioner_json', \n    'action_input': {{'query': 'images url'}}\n}}```" + "```\n\n\nIf you want to use a tool to search for information, your $JSON_BLOB might look like this:\n\n```\n{{\n    'action': 'tavily_search_results_json', \n    'action_input': {{'query': 'Example search query'}}\n}}```"  +prompt[0].prompt.template[394:]

In [6]:
from transformers import BlipProcessor, BlipForConditionalGeneration

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [7]:
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
#from pydantic import BaseModel, Field
import requests
from PIL import Image

class ImageCaptionerInput(BaseModel):
    image_url: str = Field(description="URL of the image that is to be described")


@tool("image_captioner", return_direct=True, args_schema=ImageCaptionerInput)
def image_captioner(image_url: str) -> str:
    """Provides information about the image"""
    raw_image = Image.open(requests.get(image_url, stream=True).raw).convert('RGB')
    inputs = blip_processor(raw_image, return_tensors="pt")
    out = blip_model.generate(**inputs)
    return blip_processor.decode(out[0], skip_special_tokens=True)

In [8]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
search = TavilySearchResults(max_results=1)

In [9]:
tools = [image_captioner, search]

In [10]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_structured_chat_agent
#agent = create_openai_tools_agent(llm,tools,prompt)
agent = create_structured_chat_agent(llm, tools, prompt)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [12]:
agent_executor.invoke({"input": "HI"})



> Entering new AgentExecutor chain...


Thought: The user has greeted me. I will greet them back.
Action:
```
{
  "action": "Final Answer",
  "action_input": "Hello! How can I help you today?"
}
```


> Finished chain.


{'input': 'HI', 'output': 'Hello! How can I help you today?'}

In [13]:
agent_executor.invoke({"input": "Cual es el tiempo en Santiago de Chile ahora?"})



> Entering new AgentExecutor chain...

Thought: The user is asking about the current weather in Santiago, Chile. I can use a search tool to find this information.
Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "current weather in Santiago, Chile"
  }
}
```
[{'url': 'https://en.mercopress.com/2024/02/03/heat-wave-grips-argentina-chile-uruguay-paraguay-and-brazil', 'content': 'Chile, Uruguay, and Paraguay with temperatures reaching 48 degrees Celsius and forecast to remain around or above 40  - 19:30 UTC Heat wave grips Argentina, Chile, Uruguay, Paraguay and Brazil  MercoPress, en Español Montevideo, February 3rd 2024 - 19:30 UTC  do Sul (south), according to the meteorological information agency MetSul.The unusually hot weather is expected to stay around at least for a week. Media reports all across the Southern Cone Friday underlined the heat wave affecting Argentina, Brazil, Chile, Uruguay ...'}]
Action:
```
{
  "action": "Final Answer",
  

{'input': 'Cual es el tiempo en Santiago de Chile ahora?',
 'output': 'The current weather in Santiago, Chile is very hot, with temperatures reaching 48 degrees Celsius and forecast to remain around or above 40 degrees for at least a week.'}

In [14]:
agent_executor.invoke({"input": "Que hay en esta imagen https://www.barcelo.com/guia-turismo/wp-content/uploads/2020/06/lago-di-como.jpg"})



> Entering new AgentExecutor chain...

Thought: I need to describe the image. I will use the image_captioner tool.
Action:
```
{
  "action": "image_captioner",
  "action_input": {
    "image_url": "https://www.barcelo.com/guia-turismo/wp-content/uploads/2020/06/lago-di-como.jpg"
  }
}
```


/Users/clemente/micromamba/envs/py311/lib/python3.11/site-packages/transformers/generation/utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a lake with a boat in it


> Finished chain.


{'input': 'Que hay en esta imagen https://www.barcelo.com/guia-turismo/wp-content/uploads/2020/06/lago-di-como.jpg',
 'output': 'a lake with a boat in it'}